In [ ]:
!pip install ucimlrepo

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Load the dataset
from ucimlrepo import fetch_ucirepo

# fetch dataset
htru2 = fetch_ucirepo(id=372)

# data (as pandas dataframes)
X = htru2.data.features
y = htru2.data.targets

y = y.values

# Normalize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)


In [ ]:
X_test.shape

(5370, 8)

In [ ]:
from imblearn.combine import SMOTEENN

# SMOTEENN
smoteenn = SMOTEENN(random_state=42)
X_resampled, y_resampled = smoteenn.fit_resample(X_train, y_train)


In [ ]:
# Handle outliers using IQR
# def remove_outliers_iqr(X, y):
#     Q1 = np.percentile(X, 25, axis=0)
#     Q3 = np.percentile(X, 75, axis=0)
#     IQR = Q3 - Q1
#     lower_bound = Q1 - 1.5 * IQR
#     upper_bound = Q3 + 1.5 * IQR
#     mask = np.all((X >= lower_bound) & (X <= upper_bound), axis=1)
#     return X[mask], y[mask]

# X_resampled, y_resampled = remove_outliers_iqr(X_resampled, y_resampled)


In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Convert the data to PyTorch tensors
train_dataset = TensorDataset(torch.tensor(X_resampled, dtype=torch.float32), torch.tensor(y_resampled, dtype=torch.float32).view(-1, 1))
test_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32).view(-1, 1))

# Create DataLoaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define the ANN model
class SimpleANN(nn.Module):
    def __init__(self, input_dim, hidden_layers, output_dim):
        super(SimpleANN, self).__init__()
        layers = []
        layers.append(nn.Linear(input_dim, hidden_layers[0]))
        layers.append(nn.ReLU())
        for i in range(len(hidden_layers) - 1):
            layers.append(nn.Linear(hidden_layers[i], hidden_layers[i + 1]))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(hidden_layers[-1], output_dim))
        layers.append(nn.Sigmoid())
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

# Initialize the model, loss function, and optimizer
hidden_layers = [16, 16]
learning_rate = 0.001
num_epochs = 50

model = SimpleANN(input_dim=X_resampled.shape[1], hidden_layers=hidden_layers, output_dim=1)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [1/50], Loss: 0.0001
Epoch [2/50], Loss: 0.0000
Epoch [3/50], Loss: 0.0469
Epoch [4/50], Loss: 0.0132
Epoch [5/50], Loss: 0.0034
Epoch [6/50], Loss: 0.0020
Epoch [7/50], Loss: 0.0027
Epoch [8/50], Loss: 0.0088
Epoch [9/50], Loss: 0.0018
Epoch [10/50], Loss: 0.0007
Epoch [11/50], Loss: 0.0053
Epoch [12/50], Loss: 0.0011
Epoch [13/50], Loss: 0.0031
Epoch [14/50], Loss: 0.0689
Epoch [15/50], Loss: 0.0052
Epoch [16/50], Loss: 0.1981
Epoch [17/50], Loss: 0.0007
Epoch [18/50], Loss: 0.0000
Epoch [19/50], Loss: 0.0000
Epoch [20/50], Loss: 0.0000
Epoch [21/50], Loss: 0.0000
Epoch [22/50], Loss: 0.0523
Epoch [23/50], Loss: 0.0008
Epoch [24/50], Loss: 0.0000
Epoch [25/50], Loss: 0.0000
Epoch [26/50], Loss: 0.0000
Epoch [27/50], Loss: 0.0000
Epoch [28/50], Loss: 0.0033
Epoch [29/50], Loss: 0.0000
Epoch [30/50], Loss: 0.0000
Epoch [31/50], Loss: 0.0014
Epoch [32/50], Loss: 0.0044
Epoch [33/50], Loss: 0.0320
Epoch [34/50], Loss: 0.3708
Epoch [35/50], Loss: 0.0121
Epoch [36/50], Loss: 0.0228
E

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Convert the data to PyTorch tensors
train_dataset = TensorDataset(torch.tensor(X_resampled, dtype=torch.float32), torch.tensor(y_resampled, dtype=torch.float32).view(-1, 1))
test_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32).view(-1, 1))

# Create DataLoaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define the ANN model
class SimpleANN(nn.Module):
    def __init__(self, input_dim, hidden_layers, output_dim):
        super(SimpleANN, self).__init__()
        layers = []
        layers.append(nn.Linear(input_dim, hidden_layers[0]))
        layers.append(nn.ReLU())
        for i in range(len(hidden_layers) - 1):
            layers.append(nn.Linear(hidden_layers[i], hidden_layers[i + 1]))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(hidden_layers[-1], output_dim))
        layers.append(nn.Sigmoid())
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

# Initialize the model, loss function, and optimizer
hidden_layers = [16, 16]
learning_rate = 0.0001
num_epochs = 10

model = SimpleANN(input_dim=X_resampled.shape[1], hidden_layers=hidden_layers, output_dim=1)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [1/10], Loss: 0.6364
Epoch [2/10], Loss: 0.5199
Epoch [3/10], Loss: 0.3391
Epoch [4/10], Loss: 0.3079
Epoch [5/10], Loss: 0.1692
Epoch [6/10], Loss: 2.6235
Epoch [7/10], Loss: 0.0633
Epoch [8/10], Loss: 0.0513
Epoch [9/10], Loss: 0.0132
Epoch [10/10], Loss: 0.3137


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

# Evaluate the model
model.eval()
y_true = []
y_pred = []
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        predicted = (outputs > 0.5).float()
        y_true.extend(labels.numpy())
        y_pred.extend(predicted.numpy())

y_true = np.array(y_true).flatten()
y_pred = np.array(y_pred).flatten()

# Calculate performance metrics
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")


Accuracy: 0.9652
F1 Score: 0.8292
Recall: 0.9228
Precision: 0.7529


In [ ]:
# #Accuracy: 0.9652
# F1 Score: 0.8292
# Recall: 0.9228
# Precision: 0.7529
# # Save the model to a local file
# model_path = "simple_model.pth"
# torch.save(model.state_dict(), model_path)
# print(f"Model saved to {model_path}")

Model saved to simple_model.pth


In [ ]:
X_tensor = torch.tensor(X_test, dtype=torch.float32)
y_tensor = torch.tensor(y_test)

In [ ]:
y_tensor

tensor([[1],
        [0],
        [0],
        ...,
        [1],
        [0],
        [1]])

In [ ]:
X_tensor

tensor([[-1.9916, -2.0356,  2.1439,  ...,  1.1195, -1.2136, -0.9122],
        [ 1.0497, -0.4913, -0.5571,  ..., -0.4391,  0.1941, -0.0761],
        [-0.5486,  0.3839,  0.0905,  ...,  3.7518, -1.8309, -1.0012],
        ...,
        [-2.6333, -2.5016,  3.8377,  ...,  1.6927, -1.2896, -0.9367],
        [ 0.4434, -0.4269, -0.3947,  ..., -0.6795,  1.0965,  0.8745],
        [-3.0334, -1.2436,  3.4932,  ...,  1.6391, -1.4486, -0.9604]])

In [ ]:
p = model_load(X_tensor)

In [ ]:
p.round()

tensor([[1.],
        [0.],
        [1.],
        ...,
        [1.],
        [0.],
        [1.]], grad_fn=<RoundBackward0>)

In [ ]:
import torch

# Define the path in Google Drive
model_path = '/content/drive/My Drive/simple_model.pth'

# Load the entire model
model1 = torch.load(model_path)
#model.eval()  # Set the model to evaluation mode

print("Model loaded successfully.")


Model loaded successfully.


In [ ]:
# Save the model to a local file
model_path = "Entire_model.pth"
torch.save(model.state_dict(), model_path)
print(f"Entire model saved to {model_path}")


Entire model saved to Entire_model.pth


In [ ]:
# Define the path in Google Drive
drive_model_path = '/content/drive/My Drive/Entire_model.pth'

# Move the local file to Google Drive
!cp simple_model.pth "$drive_model_path"
print(f"Model uploaded to {drive_model_path}")


Model uploaded to /content/drive/My Drive/Entire_model.pth


In [ ]:
import torch

# Define the path in Google Drive
model_path = '/content/drive/My Drive/Entire_model.pth'

# Load the entire model
#state_dict = torch.load(model_path)
#model = SimpleANN(input_dim=X_resampled.shape[1], hidden_layers=hidden_layers, output_dim=1)
model.load_state_dict(torch.load(model_path))

model.eval()  # Set the model to evaluation mode

print("Model loaded successfully from the saved model.")


Model loaded successfully from the saved model.
